In [ ]:
# ====== الخطوة 1: رفع الموديل من جهازك ثم تحميله ======
from google.colab import files
from tensorflow.keras.models import load_model
import tensorflow as tf

# 1) رفع الملف من جهازك
uploaded = files.upload()  # سيظهر لك زر اختيار ملف

# 2) الحصول على اسم الملف المرفوع
model_filename = list(uploaded.keys())[0]

# 3) تحميل الموديل
model = load_model(model_filename)
class_names = ["Fractured", "Non-Fractured"]  # عدّل حسب فئاتك
print("✅ تم تحميل الموديل بنجاح:", model_filename)

# (اختياري) تأكد من وجود GPU
print("GPU available:", tf.config.list_physical_devices('GPU'))

Saving fracture_model_final.keras to fracture_model_final.keras
✅ تم تحميل الموديل بنجاح: fracture_model_final.keras
GPU available: []


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 98 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from PIL import Image
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input

def preprocess_image(img: Image.Image, target_size=(300,300)):
    """
    تأخذ صورة PIL وتعيد مصفوفة جاهزة للموديل
    """
    # تحويل إلى RGB
    img = img.convert("RGB")
    # تغيير الحجم إلى الحجم الذي دربت عليه
    img = img.resize(target_size)
    # تحويل إلى numpy array
    x = np.array(img)
    # إضافة بعد batch
    x = np.expand_dims(x, axis=0)
    # تطبيق preprocess_input الخاص بـ EfficientNet
    x = preprocess_input(x)
    return x


In [ ]:
def predict_image(img: Image.Image):
    """
    تأخذ صورة PIL وتعيد dict فيه اسم الفئة واحتمالية التنبؤ
    """
    # 1. تحويل الصورة لصيغة الموديل
    x = preprocess_image(img, target_size=(300,300))

    # 2. تنبؤ
    pred = model.predict(x)[0][0]

    # 3. تحديد النتيجة بناءً على الاحتمال
    if pred >= 0.5:
        result_class = class_names[0]  # مكسورة
    else:
        result_class = class_names[1]  # غير مكسورة

    return {
        "class": result_class,
        "probability": float(pred)
    }


In [ ]:
from PIL import Image

# رفع صورة من جهازك للتجربة
from google.colab import files
uploaded = files.upload()
test_img_path = list(uploaded.keys())[0]

# فتح الصورة
img = Image.open(test_img_path)

# التنبؤ
result = predict_image(img)
print(result)


Saving good.jpg to good (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
{'class': 'Fractured', 'probability': 0.9761932492256165}


In [ ]:
!pip install fastapi uvicorn pyngrok --quiet


In [ ]:
!pip install fastapi uvicorn pyngrok --quiet  # pyngrok يمكن تجاهله لاحقاً

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from PIL import Image
import io
import nest_asyncio
import uvicorn

app = FastAPI()

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = Image.open(io.BytesIO(contents))
    result = predict_image(img)
    return JSONResponse(content=result)


In [ ]:
!apt-get install cloudflared -qq


E: Unable to locate package cloudflared


In [ ]:
# import subprocess
# import threading

# # السماح بتشغيل السيرفر في كولاب
# nest_asyncio.apply()

# # تشغيل السيرفر FastAPI في الخلفية
# def run_uvicorn():
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# threading.Thread(target=run_uvicorn).start()

# # إنشاء نفق cloudflared لإتاحة رابط خارجي
# def run_cloudflared():
#     subprocess.run(["cloudflared", "tunnel", "--url", "http://localhost:8000"])

# threading.Thread(target=run_cloudflared).start()
import subprocess
import threading
import time

def run_cloudflared():
    process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:8000"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True
    )
    link_found = False
    for line in process.stdout:
        print(line, end="")
        if "trycloudflare.com" in line and not link_found:
            # استخراج الرابط من السطر
            url = line.split(" ")[-1].strip()
            print("\n✅ رابط الـAPI الخارجي:", url)
            link_found = True

# تشغيل في Thread حتى لا يوقف الخلية
threading.Thread(target=run_cloudflared).start()


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from PIL import Image
import io
from fastapi.testclient import TestClient

# === تعريف التطبيق ===
app = FastAPI()

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = Image.open(io.BytesIO(contents))
    # دالة التنبؤ (فرضًا لديك predict_image جاهزة)
    result = predict_image(img)
    return JSONResponse(content=result)

# === تشغيل TestClient داخل Colab ===
client = TestClient(app)

# === مثال رفع صورة للتجربة ===
file_path = "test_fracture.jpg"  # صورة موجودة في Colab
with open(file_path, "rb") as f:
    files = {"file": ("test.jpg", f, "image/jpeg")}
    response = client.post("/predict", files=files)

print("نتيجة التنبؤ:", response.json())


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3030601209.py", lin

FileNotFoundError: [Errno 2] No such file or directory: 'test_fracture.jpg'

In [1]:
# ======= خلية واحدة شاملة =======
# تثبيت المتطلبات
!pip install fastapi uvicorn pyngrok --quiet
!apt-get install cloudflared -qq

import os, threading, subprocess, io
from google.colab import drive, files
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image
import numpy as np
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse

# ===== 2️⃣ دوال المعالجة والتنبؤ =====
def preprocess_image(img: Image.Image, target_size=(300,300)):
    img = img.convert("RGB")
    img = img.resize(target_size)
    x = np.array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def predict_image(img: Image.Image):
    x = preprocess_image(img)
    pred = model.predict(x)[0][0]
    if pred >= 0.5:
        result_class = class_names[1]
    else:
        result_class = class_names[0]
    return {"class": result_class, "probability": float(pred)}

# ===== 3️⃣ رفع صورة للتجربة =====
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# ===== 4️⃣ إعداد FastAPI =====
app = FastAPI()

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = Image.open(io.BytesIO(contents))
    result = predict_image(img)
    return JSONResponse(content=result)

# ===== 5️⃣ إغلاق أي عملية قديمة على port 8050 =====
!fuser -k 8050/tcp

# ===== 6️⃣ تشغيل FastAPI و cloudflared =====
nest_asyncio.apply()

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8050)

def run_cloudflared():
    process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:8050"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True
    )
    link_found = False
    for line in process.stdout:
        print(line, end="")
        if "trycloudflare.com" in line and not link_found:
            url = line.split(" ")[-1].strip()
            print("\n✅ رابط الـAPI الخارجي:", url)
            link_found = True

threading.Thread(target=run_uvicorn).start()
threading.Thread(target=run_cloudflared).start()


E: Unable to locate package cloudflared


KeyboardInterrupt: 

In [ ]:
from fastapi.testclient import TestClient

client = TestClient(app)

file_path = "44 (2).jpg"  # اسم الصورة التي رفعتها
with open(file_path, "rb") as f:
    files = {"file": ("44.jpg", f, "image/jpeg")}
    response = client.post("/predict", files=files)

print(response.json())


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
{'class': 'Non-Fractured', 'probability': 0.9656153917312622}
